In [1]:
import altair as alt
import pandas as pd
from omegaconf import OmegaConf

from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

import _report_visuals_utils
import viz_data_prep

readable_dict = OmegaConf.load("readable2.yml")

In [2]:
from omegaconf import OmegaConf
readable_dict = OmegaConf.load("readable2.yml")


In [3]:
FILE = GTFS_DATA_DICT.digest_tables.route_schedule_vp

# some of the portfolio grain can be dealt with
# but separate out the renaming/replacing/subsetting to separate script

df = pd.read_parquet(
    f"{RT_SCHED_GCS}{FILE}.parquet",
    filters = [[
        ("portfolio_organization_name", "==", "City of West Hollywood"),
        ("recent_combined_name", "in", ["Cityline Local-East", "Cityline Local-West"])
    ]]
).pipe(
    viz_data_prep.data_wrangling_for_visualizing,
    viz_data_prep.route_direction_cols_for_viz,
    viz_data_prep.readable_col_names
)

In [4]:
df.head(2)

,Direction (0/1),Period,Average Scheduled Service (trip minutes),# Scheduled Trips,# Realtime Trips,Date,Route,Direction,# Minutes with 1+ VP per Minute,# Minutes with 2+ VP per Minute,...,# Late Trips,Average VP per Minute,% VP within Scheduled Shape,% Scheduled Trip w/ 1+ VP/Minute,% Scheduled Trip w/ 2+ VP/Minute,Realtime versus Scheduled Service Ratio,Speed (MPH),Portfolio Organization Name,Headway (Minutes),sched_rt_category
0,0,All Day,52.73,15,11,2023-03-15,Cityline Local-East,Eastbound,455,249,...,3,1.19,95.0,78.2,42.8,1.14,8.82,City of West Hollywood,96.77,schedule_and_vp
1,0,Offpeak,52.73,9,7,2023-03-15,Cityline Local-East,Eastbound,252,145,...,1,1.21,98.0,66.3,38.2,0.98,9.44,City of West Hollywood,107.14,schedule_and_vp


In [5]:
from IPython.display import HTML, display

# Set drop down menu to be on the upper right for the charts
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

In [6]:
def route_filter(df):
    routes_list = df["Route"].unique().tolist()

    route_dropdown = alt.binding_select(
        options=routes_list,
        name="Routes: ",
    )
    # Column that controls the bar charts
    xcol_param = alt.selection_point(
        fields=["Route"], value=routes_list[0], bind=route_dropdown
    )
    
    
    chart1 = (
        _report_visuals_utils.sample_spatial_accuracy_chart(df[df.Period=="All Day"])
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    ) 

    chart2 = (
        _report_visuals_utils.sample_avg_scheduled_min_chart(df[df.Period=="All Day"])
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    
    )
    
    chart_list = [
        chart1,
        chart2
    ]
    chart = alt.vconcat(*chart_list)

    
    return chart
 

In [7]:
route_filter(df)

alt.VConcatChart(...)

## VP per Minute

In [8]:
def vp_per_minute_chart(df: pd.DataFrame) -> alt.Chart:
    specific_chart_dict = readable_dict.vp_per_min_graph
    ruler = _report_visuals_utils.ruler_chart(df, 3)
    
    bar = _report_visuals_utils.bar_chart(
        x_col = "Date", 
        y_col = "Average VP per Minute",
        color_col = "Average VP per Minute",
        color_scheme = [*specific_chart_dict.colors], 
        tooltip_cols = [*specific_chart_dict.tooltip], 
        date_format="%b %Y"
    )
    
    # write this way so that the df is inherited by .facet
    chart = alt.layer(bar, ruler, data = df).properties(width=200, height=250)
    chart = chart.facet(
        column=alt.Column(
            "Direction:N",
        )
    ).properties(
        title={
            "text": specific_chart_dict.title,
            "subtitle": specific_chart_dict.subtitle,
        }
    )
    return chart

In [9]:
vp_per_minute_chart(df[df.Period=="All Day"])

alt.FacetChart(...)

## Text Tables

## Timeliness

In [10]:
df.columns

Index(['Direction (0/1)', 'Period', 'Average Scheduled Service (trip minutes)',
       '# Scheduled Trips', '# Realtime Trips', 'Date', 'Route', 'Direction',
       '# Minutes with 1+ VP per Minute', '# Minutes with 2+ VP per Minute',
       '# Early Arrival Trips', '# On-Time Trips', '# Late Trips',
       'Average VP per Minute', '% VP within Scheduled Shape',
       '% Scheduled Trip w/ 1+ VP/Minute', '% Scheduled Trip w/ 2+ VP/Minute',
       'Realtime versus Scheduled Service Ratio', 'Speed (MPH)',
       'Portfolio Organization Name', 'Headway (Minutes)',
       'sched_rt_category'],
      dtype='object')

In [11]:
def timeliness_trips(df: pd.DataFrame):
    """
    Reshape dataframe for the charts that illustrate
    how timely a route's trips are. 
    """
    melted_df = df.melt(
    id_vars=[
        "Date",
        'Portfolio Organization Name',
        "Route",
        "Period",
        "Direction",
        'Direction (0/1)',
        "# Realtime Trips"
    ],
    value_vars=[
        "# Early Arrival Trips",
        "# On-Time Trips",
        "# Late Trips",
    ],
    )

    melted_df["Percentage"] = (melted_df.value / melted_df["# Realtime Trips"]) * 100
    
    return melted_df

In [12]:
timeliness_df = timeliness_trips(df)

In [13]:
timeliness_df.head(2)

,Date,Portfolio Organization Name,Route,Period,Direction,Direction (0/1),# Realtime Trips,variable,value,Percentage
0,2023-03-15,City of West Hollywood,Cityline Local-East,All Day,Eastbound,0,11,# Early Arrival Trips,1,9.090909
1,2023-03-15,City of West Hollywood,Cityline Local-East,Offpeak,Eastbound,0,7,# Early Arrival Trips,1,14.285714


In [16]:
def timeliness_chart(df) -> alt.Chart:
    
    # Reshape dataframe from wide to long
    df2 = timeliness_trips(df)
    
    specific_chart_dict = readable_dict.timeliness_trips_graph

    chart = _report_visuals_utils.line_chart(
        df=df2,
        x_col="Date",
        y_col="Percentage",
        color_col="variable",
        color_scheme = [*specific_chart_dict.colors],
        tooltip_cols = [*specific_chart_dict.tooltip],
    ).properties(width=200, height=250)

    chart = chart.facet(
        column=alt.Column(
            "Direction:N",
        )
    ).properties(
        title={
            "text": specific_chart_dict.title,
            "subtitle": specific_chart_dict.subtitle,
        }
    )
    return chart

In [18]:
timeliness_chart(df[df.Period == "All Day"])

alt.FacetChart(...)

## Total Scheduled Trips

In [ ]:
def total_scheduled_trips_chart(df: pd.DataFrame) -> alt.Chart:
    specific_chart_dict = readable_dict.n_scheduled_graph
    
    chart = _report_visuals_utils.bar_chart(
        x_col = "Date:T", 
        y_col = "# Scheduled Trips",
        color_col = "Period:N",
        color_scheme = [*specific_chart_dict.colors], 
        tooltip_cols = [*specific_chart_dict.tooltip], 
        date_format="%b %Y"
    )
    
    chart = alt.layer(chart, data = df)
    
    # write this way so that the df is inherited by .facet
    chart = _report_visuals_utils.configure_chart(
        chart,
        width = 400, height = 250, 
        title = specific_chart_dict.title, 
        subtitle = specific_chart_dict.subtitle
    )    
    return chart

In [ ]:
total_scheduled_trips_chart(df[df.Period !="All Day"])

## Frequency

In [ ]:
def headway_chart(df: pd.DataFrame) -> alt.Chart:
       
    specific_chart_dict = readable_dict.frequency_graph
    
    chart = _report_visuals_utils.bar_chart(
        x_col = "Date:T", 
        y_col = "Headway (Minutes)",
        color_col = "Headway (Minutes):N",
        color_scheme = [*specific_chart_dict.colors], 
        tooltip_cols = [*specific_chart_dict.tooltip], 
        date_format="%b %Y"
    )
    
    chart = alt.layer(chart, data = df)
    
    chart = chart.facet(
        column=alt.Column(
            "Direction:N",
        )
    ).properties(
        title={
            "text": specific_chart_dict.title,
            "subtitle": specific_chart_dict.subtitle,
        }
    )
    return chart

In [ ]:
headway_chart(df[(df['Period'] == "Peak")])

## Speed MPH 

In [ ]:
def speed_chart(df) -> alt.Chart:
    specific_chart_dict = readable_dict.speed_graph

    chart = _report_visuals_utils.line_chart(
        df=df,
        x_col="Date",
        y_col="Speed (MPH)",
        color_col="Period",
        color_scheme = [*specific_chart_dict.colors],
        tooltip_cols = [*specific_chart_dict.tooltip],
    ).properties(width=200, height=250)

    chart = chart.facet(
        column=alt.Column(
            "Direction:N",
        )
    ).properties(
        title={
            "text": specific_chart_dict.title,
            "subtitle": specific_chart_dict.subtitle,
        }
    )
    return chart

In [ ]:
speed_chart(df)

## % of Scheduled Trip with 1+/2+ VPs